In [1]:
from IPython.display import HTML
from IPython.display import Image 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Assignment 4 - Part 1 | Digit Recognizer
David Thor MSDS 422 - Practical Machine Learning


# Libraries

In [6]:
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from scipy import stats
import matplotlib.gridspec as gridspec
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
import glob
from PIL import Image
import PIL
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from keras.models import load_model, save_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
#from plot_keras_history import show_history, plot_history
#import scikitplot as skplt
%matplotlib inline

In [7]:
os.getcwd()

'/Users/dthor/Desktop/Assignment 4'

# Part 0 - Importing Data



In [119]:
##Importing Data
trainDat = pd.read_csv('train.csv')
testDat =  pd.read_csv('test.csv')

In [120]:
trainDat.shape
testDat.shape

(42000, 785)

(28000, 784)

In [121]:
# checking variables to make sure label is the y variable
set(trainDat.columns).difference(set(testDat.columns))

{'label'}

In [122]:
y=trainDat.label.to_numpy(copy=True)
X=trainDat.loc[:,trainDat.columns!='label'].to_numpy(copy=True)
#Divide my 255 to scale pixels. Should only be bettwen 0 and 1
X = X/255
X.shape
y.shape

(42000, 784)

(42000,)

In [123]:
## Setting 10% for final/validation testing
TrainX, TestX, Trainy, Testy = train_test_split(X,y,
                                      stratify=y,
                                      shuffle=True,
                                      test_size=0.10,
                                      random_state=3)
TrainX.shape
TestX.shape

(37800, 784)

(4200, 784)

In [124]:
## Setting 100 cases for final test data (final_exam)
XTrain,FinalX, yTrain, Finaly = train_test_split(TrainX,Trainy,
                                                    stratify=Trainy,
                                                    test_size=100,
                                                    random_state=5)
XTrain.shape
yTrain.shape
FinalX.shape
Finaly.shape

(37700, 784)

(37700,)

(100, 784)

(100,)

In [125]:
# Saving these arrays for subsequent use.

with open('Xtrain.npy','wb') as outFile:
    np.save(outFile,XTrain)

with open('yTrain.npy','wb')as outFile:
    np.save(outFile,yTrain)

with open('TestX.npy','wb') as outFile:
    np.save(outFile,TestX)

with open('Testy.npy','wb') as outFile:
    np.save(outFile,Testy)

with open('FinalX.npy','wb') as outFile:
    np.save(outFile, FinalX)

with open('Finaly.npy','wb') as outFile:
    np.save(outFile,Finaly)



## Baseline MLP
- Hidden Layer: 1
- Activation: relu
- Because the output is multiclass classification, final/last activation: softmax and compiled the model with loss = sparse_categorical_crossentropy. I used sparse because I did not one-hot encode the output
- epics: 25
- batch size: 32

In [126]:
tf.keras.backend.clear_session()
tf.random.set_seed(9)

In [127]:
#base model set up
mlp1 = tf.keras.Sequential()
mlp1.add(tf.keras.Input(shape=(784,)))
mlp1.add(layers.Dense(32,activation='relu'))
mlp1.add(layers.Dense(10,activation='softmax'))
mlp1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                25120     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [128]:
# Compile
mlp1.compile(loss='sparse_categorical_crossentropy',
             optimizer=optimizers.RMSprop(learning_rate=1e-4),
             metrics=['acc'])

### Baseline MLP Training

In [129]:
mlp1_history=mlp1.fit(XTrain, yTrain,
        epochs=25,
        batch_size=32,
        validation_data=(TestX,Testy),
        verbose=0)


In [130]:
save_model(mlp1,'mlp1.h5') #2 minutes

### Baseline MLP Results

In [131]:
mlp1_history.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [133]:
# Observe the top 5 epics with highest validation accuracy
pd.DataFrame(mlp1_history.history).sort_values(['val_acc'],\
                                               ascending=False).iloc[:5]

,loss,acc,val_loss,val_acc
24,0.176213,0.951247,0.201408,0.940238
22,0.183600,0.949416,0.206929,0.939762
23,0.179762,0.949735,0.204688,0.939286
21,0.187452,0.947507,0.210698,0.937619
20,0.191544,0.947029,0.214906,0.937619


### Baseline Final Exam

In [134]:
FinalPreds = mlp1.predict(FinalX)
#FinalPreds[:5]
#Finaly[:5]
FinalPreds.shape
Finaly.shape

(100, 10)

(100,)

In [135]:
# This will produce the accuracy of my base line model \
##   against the 100 final exam records
accuracy_baseline_final_exam = accuracy_score(Finaly,\
                                              mlp1.predict_classes(FinalX))
print (f'Baseline Final Exam Accuracy: ', \
       accuracy_baseline_final_exam)

Baseline Final Exam Accuracy:  0.95


### MLP Model 1:
 - Hidden Layers: 2
 - Activiation Method: Relu
 - Epics: 25
 - Batch size: 32
 
 - I defined these parameters/hyperparaters because I want to produce a very controlled experiment. Most of the parameteres will actually reamin the same. I will only change the hidden layers the the activation method.

In [137]:
#First layer of 32 nodes with relu
#Second layer of 64 nodes with relu
mlp2 = tf.keras.Sequential()
mlp2.add(tf.keras.Input(shape=(784,)))
mlp2.add(layers.Dense(32,activation='relu'))
mlp2.add(layers.Dense(64,activation='relu'))
mlp2.add(layers.Dense(10,activation='softmax'))
mlp2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 27,882
Trainable params: 27,882
Non-trainable params: 0
_________________________________________________________________


In [138]:
# Compile.
mlp2.compile(loss='sparse_categorical_crossentropy',
             optimizer=optimizers.RMSprop(learning_rate=1e-4),
             metrics=['acc'])

### MLP Model 1: Training

In [139]:
mlp1_history=mlp2.fit(XTrain, yTrain,
        epochs=25,
        batch_size=32,
        validation_data=(TestX,Testy),
        verbose=0)

In [140]:
save_model(mlp2,'mlp2.h5') ## 9:59 - 10:02

In [141]:
# Observe the top 5 epics with highest validation accuracy
pd.DataFrame(mlp1_history.history).sort_values(['val_acc'],
                                               \ascending=False).iloc[:5]

,loss,acc,val_loss,val_acc
22,0.131695,0.961804,0.161347,0.953333
24,0.125436,0.964324,0.157817,0.952619
23,0.128713,0.962891,0.158566,0.951905
21,0.135023,0.960504,0.163339,0.951190
20,0.138495,0.960133,0.165528,0.948571


In [143]:
accuracy_baseline_final_exam = accuracy_score(Finaly,\
                                              mlp2.predict_classes(FinalX))
print (f'Model 1 Final Exam Accuracy: ', \
       accuracy_baseline_final_exam)

Model 1 Final Exam Accuracy:  0.95


### MLP Model 2
- Hidden Layers: 2
- Activation: Leaky relu
- All other parameters and hyperparameters are the same


In [144]:
mlp3 = tf.keras.Sequential()
mlp3.add(tf.keras.Input(shape=(784,)))
mlp3.add(layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
mlp3.add(layers.Dense(64,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
mlp3.add(layers.Dense(10,activation='softmax'))
mlp3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                650       
Total params: 27,882
Trainable params: 27,882
Non-trainable params: 0
_________________________________________________________________


In [145]:
# Compile
mlp3.compile(loss='sparse_categorical_crossentropy',
             optimizer=optimizers.RMSprop(learning_rate=1e-4),
             metrics=['acc'])

In [146]:
#Training
mlp1_history=mlp3.fit(XTrain, yTrain,
        epochs=25,
        batch_size=32,
        validation_data=(TestX,Testy),
        verbose=0) 

In [147]:
save_model(mlp3,'mlp3.h5') ## 4 minutes

# Observe the top 5 epics with highest validation accuracy
pd.DataFrame(mlp1_history.history).sort_values(['val_acc'],\
                                               ascending=False).iloc[:5]

,loss,acc,val_loss,val_acc
24,0.129382,0.962732,0.161807,0.954048
23,0.132687,0.961910,0.165106,0.952143
22,0.135631,0.961645,0.166364,0.950476
20,0.142736,0.958992,0.174332,0.950476
21,0.139368,0.959310,0.171269,0.949524


In [149]:
accuracy_baseline_final_exam = accuracy_score(Finaly,\
                                              mlp3.predict_classes(FinalX))
print (f'Model 3 Final Exam Accuracy: ', \
       accuracy_baseline_final_exam)

Model 3 Final Exam Accuracy:  0.94


### MLP Model 3
- Hidden Layers: 4
- Activiation Method: relu

In [154]:
mlp4 = tf.keras.Sequential()
mlp4.add(tf.keras.Input(shape=(784,)))
mlp4.add(layers.Dense(32,activation='relu'))
mlp4.add(layers.Dense(64,activation='relu'))
mlp4.add(layers.Dense(64,activation='relu'))
mlp4.add(layers.Dense(32,activation='relu'))
mlp4.add(layers.Dense(10,activation='softmax'))
mlp4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12 (Dense)             (None, 10)                330       
Total params: 33,802
Trainable params: 33,802
Non-trainable params: 0
_________________________________________________________________


In [155]:
# Compile
mlp4.compile(loss='sparse_categorical_crossentropy',
             optimizer=optimizers.RMSprop(learning_rate=1e-4),
             metrics=['acc'])

In [156]:
#Training
mlp1_history=mlp4.fit(XTrain, yTrain,
        epochs=25,
        batch_size=32,
        validation_data=(TestX,Testy),
        verbose=0) 

In [157]:
save_model(mlp4,'mlp4.h5') ##10:14 2 minutes

# Observe the top 5 epics with highest validation accuracy
pd.DataFrame(mlp1_history.history).sort_values(['val_acc'],\
                                               ascending=False).iloc[:5]

,loss,acc,val_loss,val_acc
23,0.093672,0.971989,0.152665,0.956429
24,0.091273,0.973289,0.152100,0.955476
22,0.096529,0.971459,0.153486,0.954762
20,0.103019,0.969735,0.156492,0.954762
21,0.099935,0.970371,0.157663,0.954048


In [158]:
accuracy_baseline_final_exam = accuracy_score(Finaly,\
                                              mlp4.predict_classes(FinalX))
print (f'Model 3 Final Exam Accuracy: ', \
       accuracy_baseline_final_exam)

Model 3 Final Exam Accuracy:  0.95


### MLP Model 4
- Hidden Layers: 4
- Activiation Method: Leaky relu

In [159]:
mlp5 = tf.keras.Sequential()
mlp5.add(tf.keras.Input(shape=(784,)))
mlp5.add(layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
mlp5.add(layers.Dense(64,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
mlp5.add(layers.Dense(64,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
mlp5.add(layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
mlp5.add(layers.Dense(10,activation='softmax'))
mlp5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 32)                25120     
_________________________________________________________________
dense_14 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_15 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_16 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_17 (Dense)             (None, 10)                330       
Total params: 33,802
Trainable params: 33,802
Non-trainable params: 0
_________________________________________________________________


In [160]:
# Compile
mlp5.compile(loss='sparse_categorical_crossentropy',
             optimizer=optimizers.RMSprop(learning_rate=1e-4),
             metrics=['acc'])

In [161]:
#Training
mlp1_history=mlp5.fit(XTrain, yTrain,
        epochs=25,
        batch_size=32,
        validation_data=(TestX,Testy),
        verbose=0) 

In [163]:
save_model(mlp5,'mlp5.h5') ## 5 minutes

# Observe the top 5 epics with highest validation accuracy
pd.DataFrame(mlp1_history.history).sort_values(['val_acc'],\
                                               ascending=False).iloc[:5]

,loss,acc,val_loss,val_acc
23,0.096378,0.971565,0.140758,0.960238
24,0.093414,0.972308,0.139156,0.960000
21,0.102547,0.968780,0.146551,0.959286
22,0.099096,0.969735,0.143127,0.958571
20,0.105979,0.968223,0.145524,0.956190


In [181]:
accuracy_baseline_final_exam = accuracy_score(Finaly,\
                                              mlp5.predict_classes(FinalX))
print (f'Model 4 Final Exam Accuracy: ', \
       accuracy_baseline_final_exam)

Model 4 Final Exam Accuracy:  0.96


## MLP Designed Experiment Results Below
- The table only shows what I changed in the designed experiment.
- There were other parameters/hyperparameters such as batch size, epochs (epics) that remained constant
- I also monitored the processing times shown in the table
- Overall, very similar results. With additional time, I could've further tune the MLP model and analyzed with additional metrics such as area under the curve. However, accuracy was suffice for this learning objective. It iis also easy to interpret and explain to non-technical individuals.

In [252]:
from tabulate import tabulate

mlp_results = [
    ["Baseline", "1", "relu", "2", "95%"],
    ["1", "2", "relu", "3", "95%"],
    ["2", "2", "relu", "3", "94%"],
    ["3", "4", "Leaky relu", "3", "95%"],
    ["4", "4", "Leaky relu", "5", "96%"]
]
 
# create header
mlp_head = ["MLP Model", "Hidden Layers", "Activation",\
            "Processing Time (minutes)", "Final Test Accuracy"]
 
# display table
#print(tabulate(mlp_results, headers=mlp_head, tablefmt="html"))

<table align="left">
<thead>
<tr><th>MLP Model  </th><th style="text-align: right;">  Hidden Layers</th><th>Activation  </th><th style="text-align: right;">  Processing Time (minutes)</th><th>Final Test Accuracy  </th></tr>
</thead>
<tbody>
<tr><td>Baseline   </td><td style="text-align: right;">              1</td><td>relu        </td><td style="text-align: right;">                          2</td><td>95%                  </td></tr>
<tr><td>1          </td><td style="text-align: right;">              2</td><td>relu        </td><td style="text-align: right;">                          3</td><td>95%                  </td></tr>
<tr><td>2          </td><td style="text-align: right;">              2</td><td>relu        </td><td style="text-align: right;">                          3</td><td>94%                  </td></tr>
<tr><td>3          </td><td style="text-align: right;">              4</td><td>Leaky relu  </td><td style="text-align: right;">                          3</td><td>95%                  </td></tr>
<tr><td>4          </td><td style="text-align: right;">              4</td><td>Leaky relu  </td><td style="text-align: right;">                          5</td><td>96%                  </td></tr>
</tbody>
</table>

## Baseline CNN

In [168]:
#Reshape the 784 back to 28x28 as CNN requires it
XTrainRSR = XTrain.reshape(-1,28,28,1)
TestXRSR = TestX.reshape(-1,28,28,1)
FinalXRSR = FinalX.reshape(-1,28,28,1)
FinalXRSR.shape

(100, 28, 28, 1)

In [172]:
tf.keras.backend.clear_session()
tf.random.set_seed(9)

In [176]:
# Baseline parameters for my CNN. \
# These were also taken from the sync session with the Xray example.
# Again, the final activiation is softmax as 
##  this is multiclass with 10 different classifications.
inputs = tf.keras.Input(shape=(28,28,1))
x=layers.Conv2D(filters=32,
                kernel_size=3,
                activation='relu')(inputs)
x=layers.MaxPooling2D()(x)
x=layers.Flatten()(x)
outputs=layers.Dense(10,activation='softmax')(x)

convModel1=models.Model(inputs=inputs,outputs=outputs)

In [177]:
convModel1.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________


In [179]:
#Compile baseline CNN
convModel1.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc'])

In [180]:
#Training. I wanted similar batch size and epochs size 
##  from the MLP models to really be able to get a great comparison
convModel1_history=convModel1.fit(XTrainRSR,yTrain,
                                 batch_size=32,
                                 epochs=25,
                                validation_data=(TestXRSR,Testy),
        verbose=0)

In [182]:
convModel1_history.history.keys() #10:47 -- 10:52: 5 minutes

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [191]:
## This code shows the accuracy of the baseline CNN model
## I had to use the np.argmax for the multiclass classification 
accuracy_baseline_final_exam = accuracy_score(Finaly,\
                            np.argmax(convModel1.predict(FinalXRSR),axis=1))
print (f'Baseline Final Exam Accuracy: ', \
       accuracy_baseline_final_exam)

## Interestingly, the accuracy was 100%...

Baseline Final Exam Accuracy:  1.0


## CNN Model 1
 - Hidden layers: 2
 - Activation: relu
 - Batch Size: 32
 - Epochs 25
 - Again, I wanted a controlled experiement with as much of the constants for a good comparison.
 - The only parameters for the designed experiment will be hidden layers and activation.

In [192]:
inputs = tf.keras.Input(shape=(28,28,1))
x=layers.Conv2D(filters=32,
                kernel_size=3,
                activation='relu')(inputs)
x=layers.MaxPooling2D()(x)
x=layers.Conv2D(filters=64,
               kernel_size=3,
               activation='relu')(x)
x=layers.MaxPool2D()(x)
x=layers.Flatten()(x)
outputs=layers.Dense(10,activation='softmax')(x)

convModel2=models.Model(inputs=inputs,outputs=outputs)

In [193]:
convModel2.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1601

In [194]:
#Compile CNN model 1
convModel2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc'])

In [195]:
#Training. All other parameters the same
convModel1_history=convModel2.fit(XTrainRSR,yTrain,
                                 batch_size=32,
                                 epochs=25,
                                validation_data=(TestXRSR,Testy),
        verbose=0)

In [196]:
convModel1_history.history.keys() #11:11 - 11:18 7 minutes

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [197]:
## This code shows the accuracy of the baseline CNN model
## I had to use the np.argmax for the multiclass classification 
accuracy_baseline_final_exam = accuracy_score\
(Finaly,np.argmax(convModel2.predict(FinalXRSR),axis=1))
print (f'CNN Model 1 Final Exam Accuracy: ', \
       accuracy_baseline_final_exam)

CNN Model 1 Final Exam Accuracy:  0.99


## CNN Model 2
- Hidden Layers: 2
- Activation: Leaky relu

In [199]:
inputs = tf.keras.Input(shape=(28,28,1))
x=layers.Conv2D(filters=32,
                kernel_size=3,
                activation=tf.keras.layers.LeakyReLU(alpha=0.01))(inputs)
x=layers.MaxPooling2D()(x)
x=layers.Conv2D(filters=64,
               kernel_size=3,
               activation=tf.keras.layers.LeakyReLU(alpha=0.01))(x)
x=layers.MaxPool2D()(x)
x=layers.Flatten()(x)
outputs=layers.Dense(10,activation='softmax')(x)

convModel3=models.Model(inputs=inputs,outputs=outputs)

In [200]:
convModel3.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1601

In [201]:
#Compile CNN model 2
convModel3.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc'])

In [202]:
#Training. All other parameters the same
convModel1_history=convModel3.fit(XTrainRSR,yTrain,
                                 batch_size=32,
                                 epochs=25,
                                validation_data=(TestXRSR,Testy),
        verbose=0)

In [203]:
convModel1_history.history.keys() #10 minutes

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [204]:
## This code shows the accuracy of the baseline CNN model
## I had to use the np.argmax for the multiclass classification 
accuracy_baseline_final_exam = accuracy_score\
(Finaly,np.argmax(convModel3.predict(FinalXRSR),axis=1))
print (f'CNN Model 2 Final Exam Accuracy: ', \
       accuracy_baseline_final_exam)

CNN Model 2 Final Exam Accuracy:  1.0


## CNN Model 3
- Hidden Layers: 3
- Activation relu
- All other parameters were constant in order to get a good comparison

In [215]:
inputs = tf.keras.Input(shape=(28,28,1))
x=layers.Conv2D(filters=32,
                kernel_size=3,
                activation='relu')(inputs)
x=layers.MaxPooling2D()(x)
x=layers.Conv2D(filters=64,
               kernel_size=3,
               activation='relu')(x)
x=layers.MaxPooling2D()(x)
x=layers.Conv2D(filters=32,
               kernel_size=3,
               activation='relu')(x)
x=layers.MaxPool2D()(x)
x=layers.Flatten()(x)
outputs=layers.Dense(10,activation='softmax')(x)

convModel4=models.Model(inputs=inputs,outputs=outputs)

In [216]:
convModel4.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 3, 3, 32)          18464     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 1, 1, 32)          0  

In [217]:
#Compile CNN model 3
convModel4.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc'])

In [218]:
#Training. All other parameters the same
convModel1_history=convModel4.fit(XTrainRSR,yTrain,
                                 batch_size=32,
                                 epochs=25,
                                validation_data=(TestXRSR,Testy),
        verbose=0)

In [219]:
convModel1_history.history.keys() #11:43 - 11:50.  7 minutes

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [220]:
## This code shows the accuracy of the baseline CNN model
## I had to use the np.argmax for the multiclass classification 
accuracy_baseline_final_exam = accuracy_score\
(Finaly,np.argmax(convModel4.predict(FinalXRSR),axis=1))
print (f'CNN Model 3 Final Exam Accuracy: ', \
       accuracy_baseline_final_exam)

CNN Model 3 Final Exam Accuracy:  0.99


## CNN Model 4
- Hidden Layers: 3
- Activation: Leaky relu

In [222]:
inputs = tf.keras.Input(shape=(28,28,1))
x=layers.Conv2D(filters=32,
                kernel_size=3,
                activation=tf.keras.layers.LeakyReLU(alpha=0.01))(inputs)
x=layers.MaxPooling2D()(x)
x=layers.Conv2D(filters=64,
               kernel_size=3,
               activation=tf.keras.layers.LeakyReLU(alpha=0.01))(x)
x=layers.MaxPooling2D()(x)
x=layers.Conv2D(filters=32,
               kernel_size=3,
               activation=tf.keras.layers.LeakyReLU(alpha=0.01))(x)
x=layers.MaxPool2D()(x)
x=layers.Flatten()(x)
outputs=layers.Dense(10,activation='softmax')(x)

convModel5=models.Model(inputs=inputs,outputs=outputs)

In [223]:
convModel5.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 3, 3, 32)          18464     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 1, 1, 32)          0  

In [224]:
#Compile CNN model 1
convModel5.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc'])

In [225]:
#Training. All other parameters the same
convModel1_history=convModel5.fit(XTrainRSR,yTrain,
                                 batch_size=32,
                                 epochs=25,
                                validation_data=(TestXRSR,Testy),
        verbose=0)

In [226]:
convModel1_history.history.keys() #8 minutes

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [227]:
## This code shows the accuracy of the baseline CNN model
## I had to use the np.argmax for the multiclass classification 
accuracy_baseline_final_exam = accuracy_score\
(Finaly,np.argmax(convModel5.predict(FinalXRSR),axis=1))
print (f'CNN Model 4 Final Exam Accuracy: ', \
       accuracy_baseline_final_exam)

CNN Model 4 Final Exam Accuracy:  0.97


# CNN and MLP Results/Summary
 - The table below shows the 10 different models I trained
 - It compares the hidden layers, activition between the two models
 - Again, I could've spent more time tuning, looking at other performance metrics (confusion matrix, AUC, etc.).
 - Given the short time, I've decided to conclude with CNN model 2 as it yeilded 100% accuracy on the final exam.
 - Also, with the small set of parameters for the designed experiment, the different processing times did not vary enough for me to make a decision on which model to select.

In [254]:
from tabulate import tabulate

model_results = [
    ["Baseline MLP", "1", "relu", "2", "95%"],
    ["MLP 1", "2", "relu", "3", "95%"],
    ["MLP 2", "2", "relu", "3", "94%"],
    ["MLP 3", "4", "Leaky relu", "3", "95%"],
    ["MLP 4", "4", "Leaky relu", "5", "96%"],
    ["Baseline CNN", "1", "relu", "7", "100%"],
    ["CNN 1", "2", "relu", "7", "99%"],
    ["CNN 2", "2", "Leaky relu", "10", "100%"],
    ["CNN 3", "3", "relu", "7", "99%"],
    ["CNN 4", "3", "Leaky relu", "7", "97%"]
]
 
# create header
model_head = ["Model", "Hidden Layers", "Activation",\
              "Processing Time (minutes)", "Final Test Accuracy"]
 
# display table
#print(tabulate(model_results, headers=model_head, tablefmt="html"))

<table align="left">
<thead>
<tr><th>Model       </th><th style="text-align: right;">  Hidden Layers</th><th>Activation  </th><th style="text-align: right;">  Processing Time (minutes)</th><th>Final Test Accuracy  </th></tr>
</thead>
<tbody>
<tr><td>Baseline MLP</td><td style="text-align: right;">              1</td><td>relu        </td><td style="text-align: right;">                          2</td><td>95%                  </td></tr>
<tr><td>MLP 1       </td><td style="text-align: right;">              2</td><td>relu        </td><td style="text-align: right;">                          3</td><td>95%                  </td></tr>
<tr><td>MLP 2       </td><td style="text-align: right;">              2</td><td>relu        </td><td style="text-align: right;">                          3</td><td>94%                  </td></tr>
<tr><td>MLP 3       </td><td style="text-align: right;">              4</td><td>Leaky relu  </td><td style="text-align: right;">                          3</td><td>95%                  </td></tr>
<tr><td>MLP 4       </td><td style="text-align: right;">              4</td><td>Leaky relu  </td><td style="text-align: right;">                          5</td><td>96%                  </td></tr>
<tr><td>Baseline CNN</td><td style="text-align: right;">              1</td><td>relu        </td><td style="text-align: right;">                          7</td><td>100%                 </td></tr>
<tr><td>CNN 1       </td><td style="text-align: right;">              2</td><td>relu        </td><td style="text-align: right;">                          7</td><td>99%                  </td></tr>
<tr><td>CNN 2       </td><td style="text-align: right;">              2</td><td>Leaky relu  </td><td style="text-align: right;">                         10</td><td>100%                 </td></tr>
<tr><td>CNN 3       </td><td style="text-align: right;">              3</td><td>relu        </td><td style="text-align: right;">                          7</td><td>99%                  </td></tr>
<tr><td>CNN 4       </td><td style="text-align: right;">              3</td><td>Leaky relu  </td><td style="text-align: right;">                          7</td><td>97%                  </td></tr>
</tbody>
</table>

## Implement/Submission
- CNN 2 was the winner. Hidden layers: 2. Activation Leaky Relu.

In [230]:
testDat =  pd.read_csv('test.csv')

In [231]:
test_final_exam=testDat.to_numpy(copy=True)
#Divide my 255 to scale pixels. Should only be bettwen 0 and 1
test_final_exam = test_final_exam/255

In [236]:
#Reshape for CNN
test_final_exam_RS = test_final_exam.reshape(-1,28,28,1)

In [246]:
y_pred = np.argmax(convModel3.predict(test_final_exam_RS),axis=1)

In [247]:
# Method to produce the submission file
def write_preds(preds, fname):
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), \
                  "Label": preds}).to_csv(fname, index=False, header=True)

In [248]:
# Write to file your test score for submission
write_preds(y_pred, "digit_submission_CNN.csv")